## Contextualized model

Let's check where the context really helps

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [30]:
%load_ext autoreload
%autoreload 2

from hatedetection import load_datasets
import glob
import json

train_dataset, dev_dataset, test_dataset = load_datasets(add_body=True)

no_context_evals = []
context_evals = []

for path in glob.glob("../evaluations/non-context*"):
    with open(path) as f:
        obj = json.load(f)
        obj["file"] = path
        no_context_evals.append(obj)

for path in glob.glob("../evaluations/context*"):
    with open(path) as f:
        obj = json.load(f)
        obj["file"] = path
        context_evals.append(obj)

print(f"We have {len(context_evals)} context evaluations")
print(f"We have {len(no_context_evals)} context evaluations")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
We have 6 context evaluations
We have 10 context evaluations


In [31]:
import pandas as pd

df_context_evals = pd.DataFrame([
    {**{"file": evaluation["file"]}, **evaluation["metrics"]} for evaluation in context_evals
])

df_context_evals

,file,eval_loss,eval_accuracy,eval_f1,eval_macro_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,init_mem_cpu_alloc_delta,init_mem_gpu_alloc_delta,init_mem_cpu_peaked_delta,init_mem_gpu_peaked_delta,test_mem_cpu_alloc_delta,test_mem_gpu_alloc_delta,test_mem_cpu_peaked_delta,test_mem_gpu_peaked_delta
0,../evaluations/context-4,0.80791,0.91025,0.68267,0.81520,0.77605,0.60935,85.04410,133.37800,51782,0,18258,0,405647,0,887233,189087744
1,../evaluations/context-3,0.84004,0.90391,0.66564,0.80477,0.74163,0.60378,172.47950,65.76400,51854,0,18258,0,409133,0,887288,189087744
2,../evaluations/context-1,0.80262,0.90505,0.66563,0.80515,0.75281,0.59655,219.52600,51.67000,51854,0,18258,0,405281,0,887225,189087744
3,../evaluations/context-2,0.81286,0.90576,0.66770,0.80640,0.75634,0.59766,217.89450,52.05700,51854,0,18258,0,405778,0,887161,189087744
4,../evaluations/context-5,0.56969,0.90602,0.67341,0.80926,0.74915,0.61157,84.75670,133.83000,51854,0,18258,0,405147,0,887224,189087744
5,../evaluations/context-6,0.89396,0.90179,0.66222,0.80238,0.72751,0.60768,88.59010,128.03900,51854,0,18258,0,405339,0,887161,189087744


In [32]:
metric_columns = ["eval_precision", "eval_recall", "eval_f1", "eval_macro_f1"]

pd.DataFrame({"mean": df_context_evals[metric_columns].mean(), "std": df_context_evals[metric_columns].std()})

,mean,std
eval_precision,0.75058,0.01613
eval_recall,0.60443,0.00623
eval_f1,0.66955,0.00741
eval_macro_f1,0.80719,0.00452


In [33]:
import pandas as pd

df_no_context_evals = pd.DataFrame([
    {**{"file": evaluation["file"]}, **evaluation["metrics"]} for evaluation in no_context_evals
])

pd.DataFrame({"mean": df_no_context_evals[metric_columns].mean(), "std": df_no_context_evals[metric_columns].std()})

,mean,std
eval_precision,0.67690,0.01804
eval_recall,0.59738,0.01516
eval_f1,0.63431,0.00511
eval_macro_f1,0.78509,0.00302


## Resultados

In [24]:
import pandas as pd
pd.options.display.max_columns = 40
pd.set_option('display.float_format', lambda x: '%.5f' % x)


df_results = pd.concat([])

In [8]:
df_results = pd.concat([df_no_context_results, df_context_results, df_full_context_results])
df_results["index"] = ["No context", "Title", "Title and Body"]
df_results.set_index("index")

,eval_loss,eval_accuracy,eval_f1,eval_macro_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,init_mem_cpu_alloc_delta,init_mem_gpu_alloc_delta,init_mem_cpu_peaked_delta,init_mem_gpu_peaked_delta,eval_mem_cpu_alloc_delta,eval_mem_gpu_alloc_delta,eval_mem_cpu_peaked_delta,eval_mem_gpu_peaked_delta
index,,,,,,,,,,,,,,,,
No context,0.65833,0.88945,0.63525,0.78505,0.66545,0.60768,71.59330,158.43700,43460,0,18258,0,233460,0,897173,69492736
Title,0.80262,0.90505,0.66563,0.80515,0.75281,0.59655,105.10900,107.91600,52521,0,18258,0,247803,0,897045,189087744
Title and Body,0.81926,0.87137,0.35585,0.64220,0.86111,0.22426,286.39620,39.60600,43213,0,18258,0,501766,0,887205,579272704


## Dev Results

In [8]:
df_results = pd.concat([df_no_context_results, df_context_results, df_full_context_results])
df_results["index"] = ["No context", "Title", "Title and Body"]
df_results.set_index("index")

,eval_loss,eval_accuracy,eval_f1,eval_macro_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,init_mem_cpu_alloc_delta,init_mem_gpu_alloc_delta,init_mem_cpu_peaked_delta,init_mem_gpu_peaked_delta,eval_mem_cpu_alloc_delta,eval_mem_gpu_alloc_delta,eval_mem_cpu_peaked_delta,eval_mem_gpu_peaked_delta
index,,,,,,,,,,,,,,,,
No context,0.50128,0.90753,0.66980,0.80802,0.73431,0.61572,38.45080,236.82200,43460,0,18258,0,200737,0,755193,69447680
Title,0.60040,0.92554,0.73883,0.84771,0.79322,0.69142,76.05850,119.72400,52465,0,18258,0,209357,0,756777,189042688
Title and Body,0.85207,0.86130,0.18778,0.55598,0.86905,0.10526,156.20100,58.29700,43325,0,18258,0,430762,0,742281,579227648


In [19]:
example = context_dataset[0]

context_tokenizer.decode(example["input_ids"])

'[CLS] [USER] ojalá se funda y cague de hambre [SEP] mora godoy cierra su escuela de tango y remata su vestuario para " poder seguir adelante " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD